
# Training model for text summarization


## 1 GPU Setup and Python Library Installation & Import 

###1.1 GPU Setup

In [1]:
import torch
# Checking GPU availability
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


### 1.2 Library *Installation*

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###1.3 Library Import

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from transformers import *
from tqdm import tqdm, trange
from ast import literal_eval

/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(


###1.4 Connceting to Google Drive

In [5]:
# run this code when running the code on Google Colab
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/Applied_ML_Project/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## 2 Data Loading and Preprocessing

In [6]:
data = pd.read_pickle("/content/drive/MyDrive/Applied_ML_Project/data_preprocessing/summary_data1.pkl")
data

,original_text,reference_summary
0,"On September 15, 2005, the Equal Employment Op...",Equal Employment Opportunity Commission brough...
1,NOTE: This is one of three identically named c...,The case was brought by a non-profit organizat...
2,"On May 11, 2006, African-American employees of...",This case was brought by African American empl...
3,Pursuant to the Civil Rights of Institutionali...,Pursuant to the Civil Rights of Institutionali...
4,"On July 30, 2015, the Freedom of the Press Fou...",A non-profit organization dealing with rights ...
...,...,...
30861,BT program to beat dialler scams BT is introd...,BT is introducing two initiatives to help beat...
30862,Spam e-mails tempt net shoppers Computer user...,A third of them read unsolicited junk e-mail a...
30863,Be careful how you code A new European direct...,This goes to the heart of the European project...
30864,US cyber security chief resigns The man makin...,Amit Yoran was director of the National Cyber ...


In [7]:
data['original_text'][0]

'On September 15, 2005, the Equal Employment Opportunity Commission (EEOC) filed suit against House of Philadelphia, Inc., on behalf of an employee who was allegedly fired because she was pregnant. Seeking monetary and injunctive relief for the employee (including economic damage, compensation for emotional harm, and punitive damages), the EEOC brought suit under Title VII of the Civil Rights Act of 1964 for unlawful discrimination on the basis of sex. The EEOC also sought to recover its costs. Via private counsel, the employee filed a motion to intervene in the suit, which was automatically granted after the period for filing objections passed without incident. The employee brought claims under Title VII and state law and sought substantially the same relief as the EEOC, except that the complaint specifically sought reinstatement. Eventually the parties came to a settlement agreement, which the Court (Judge Kristi K. DuBose) entered as a consent decree on Jan 10, 2009. The terms of th

In [8]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/5bf53e1f76b1430d9302d735c613c5f5677e32a6/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty

In [9]:
import datasets
from datasets import Dataset, DatasetDict
tds = Dataset.from_pandas(data)


In [10]:
tds = tds.train_test_split(test_size=0.2)

In [11]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["original_text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["reference_summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
tokenized_tds = tds.map(preprocess_function, batched=True)

Map:   0%|          | 0/24692 [00:00<?, ? examples/s]

Map:   0%|          | 0/6174 [00:00<?, ? examples/s]

In [13]:

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [14]:
!pip install evaluate
!pip install rouge_score
import evaluate

rouge = evaluate.load("rouge")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=62acb5dfcaf5eb82e7fe8362158b06d4508555c8f9772c928c6fa5e304b895b7
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [15]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [35]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/5bf53e1f76b1430d9302d735c613c5f5677e32a6/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
   

In [39]:
# !pip install huggingface_hub transformers
from huggingface_hub import notebook_login, create_repo

notebook_login()

# Result commit https://huggingface.co/osanseviero/test_bug/commit/42718dae9d039325f55143cbff23f57c5098eb7d

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [45]:
training_args = Seq2SeqTrainingArguments(
    output_dir="Saish/policy_summarization_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_tds["train"],
    eval_dataset=tokenized_tds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/Saish/policy_summarization_model into local empty directory.
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: original_text, reference_summary. If original_text, reference_summary are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` t

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.875600,1.639824,0.221800,0.160300,0.207100,0.207200,18.986900
2,1.786200,1.573044,0.225300,0.164200,0.211200,0.211300,18.986400
3,1.743800,1.548427,0.227000,0.166000,0.212900,0.213000,18.984600
4,1.725600,1.539399,0.227600,0.167000,0.213600,0.213700,18.984100


Saving model checkpoint to Saish/policy_summarization_model/checkpoint-500
Configuration saved in Saish/policy_summarization_model/checkpoint-500/config.json
Configuration saved in Saish/policy_summarization_model/checkpoint-500/generation_config.json
Model weights saved in Saish/policy_summarization_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in Saish/policy_summarization_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in Saish/policy_summarization_model/checkpoint-500/special_tokens_map.json
tokenizer config file saved in Saish/policy_summarization_model/tokenizer_config.json
Special tokens file saved in Saish/policy_summarization_model/special_tokens_map.json
Saving model checkpoint to Saish/policy_summarization_model/checkpoint-1000
Configuration saved in Saish/policy_summarization_model/checkpoint-1000/config.json
Configuration saved in Saish/policy_summarization_model/checkpoint-1000/generation_config.json
Model weights saved in Saish/p

TrainOutput(global_step=6176, training_loss=1.8246807582637807, metrics={'train_runtime': 7085.6535, 'train_samples_per_second': 13.939, 'train_steps_per_second': 0.872, 'total_flos': 2.673487809557299e+16, 'train_loss': 1.8246807582637807, 'epoch': 4.0})